In [1]:
import urllib, json, os, datetime
!pip install unidecode
from unidecode import unidecode
!pip install unicodecsv
import unicodecsv as csv
import urllib.request


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 235 kB 5.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for unicodecsv: filename=unicodecsv-0.14.1-py3-none-any.whl size=10768 sha256=49fd393ecf97540bf06b1a1629f944a009e3bdf88fe5ec35d75fe2c6f0ec18f5
  Stored in directory: /root/.cache/pip/wheels/1a/f4/8a/a5024fb77b32ed369e5c409081e5f00fbe3b92fdad653f6e69
Successfully built unicodecsv


In [6]:
#CHANGE THESE SETTINGS!
saveDirectory = '/content'
boardLetter = 'pol'

print ("The current working directory is " + saveDirectory)  

#First we'll try to create a folder for the board.
path = boardLetter
print ("Attempting to create directory for board at %s" % path)
try:  
	os.mkdir(path)
except OSError:  
	print ("Creation of the directory %s failed  - possible directory already exists" % path)
else:  
	print ("Successfully created the directory %s" % path)

The current working directory is /content
Attempting to create directory for board at pol
Successfully created the directory pol


In [7]:
#Get the 4chan board catalog JSON file and open it
url = "https://a.4cdn.org/" + boardLetter + "/catalog.json"
response = urllib.request.urlopen(url)
threadCatalog = json.loads(response.read())

In [8]:
print("BEGINNING 4CHAN FRONT PAGE SCRAPE")
print("Current board: " + boardLetter)

downloadCounter = 0

#Only look at the front page
frontPage = threadCatalog[0]['threads']
#print frontPage[0]['com']

#Create a file to list all threads we've analyzed
allThreadFile = boardLetter + "/frontPage-" + str(datetime.datetime.now()) + ".csv"
with open(allThreadFile, 'wb') as thread_file:
			thread_writer = csv.writer(thread_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_ALL)
			thread_writer.writerow(["timeStamp", "lastScraped", "posterID", "name", "postID", "subjectText", "commentText"])

print ("Now parsing front page threads...")
#There are always 20 threads on the front page. So we'll loop through and read each one
i = 0

while i < 20:
	print ("STARTING NEW THREAD - #" + str(frontPage[i]['no']))
	
	#Open the thread listing file, and write the information for the current thread
	#First we make some failsafes - if not all the information is filled in.
	if 'sub' in frontPage[i]:
		subjectText = frontPage[i]['sub']
	else:
		subjectText = "No Subject Text Provided"
	if 'com' in frontPage[i]:
		commentText = frontPage[i]['com']
	else:
		commentText = "No Comment Text Provided"
	with open(allThreadFile, 'wb') as thread_file:
			thread_writer = csv.writer(thread_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_ALL)
			try:
				thread_writer.writerow([frontPage[i]['now'], datetime.datetime.now(), frontPage[i]['id'], frontPage[i]['name'], frontPage[i]['no'], subjectText, commentText])
			except:
				thread_writer.writerow([frontPage[i]['now'], datetime.datetime.now(), "No ID", frontPage[i]['name'], frontPage[i]['no'], subjectText, commentText])

	
	# First we create a new directory for the thread
	path = boardLetter + "/" + str(frontPage[i]['no']) + " - " + frontPage[i]['semantic_url']
	print ("Attempting to create directory for thread %s" % path)
	try:  
	    os.mkdir(path)
	except OSError:  
	    print ("Creation of the directory %s failed  - possible directory already exists" % path)
	else:  
	    print ("Successfully created the directory %s" % path)

	#Now we create a CSV File for the individual thread
	with open(path + "/thread.csv", 'wb') as thread_file:
			thread_writer = csv.writer(thread_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_ALL)
			thread_writer.writerow(["timeStamp", "posterID", "name", "postID", "subjectText", "commentText", "filename"])

	#Get the individual thread JSON file from 4chan and open it
	url = "https://a.4cdn.org/" + boardLetter + "/thread/" + str(frontPage[i]['no']) + ".json"
	response = urllib.request.urlopen(url)
	individualThread = json.loads(response.read())
	
	#print individualThread['posts']
	
	#Next, we loop through every post in this thread, and start gathering information. I use j to count which post (within the thread) I'm on
	j = 0
	for post in individualThread['posts']:
		#print post
		print ("Now processing post " + str(individualThread['posts'][j]['no']))
		timeStamp = individualThread['posts'][j]['now']
		name = individualThread['posts'][j]['name']
		try:
			posterID = individualThread['posts'][j]['id']
		except:
			posterID = "No ID"
		postID = individualThread['posts'][j]['no']
		if 'sub' in individualThread['posts'][j]:
			subjectText = individualThread['posts'][j]['sub']
		else:
			subjectText = "No Subject Text Provided"
		if 'com' in individualThread['posts'][j]:
			commentText = individualThread['posts'][j]['com']
		else:
			commentText = "No Comment Text Provided"
		#Was there an image posted? Let's snag it!
		if 'tim' in individualThread['posts'][j]:
			OGfilename = unidecode(individualThread['posts'][j]['filename'])
			#OGfilename = unidecode(OGfilename)
			renamedFile = str(individualThread['posts'][j]['tim'])
			fileExtension = str(individualThread['posts'][j]['ext'])
			filename = OGfilename + " - " + renamedFile + fileExtension
			postFile = urllib.request.urlopen(url)
			try:
				postFile.retrieve("https://i.4cdn.org/" + boardLetter + "/" + renamedFile + fileExtension, saveDirectory + "/" + boardLetter + "/" + str(frontPage[i]['no']) + " - " + frontPage[i]['semantic_url'] + "/" + filename)
				downloadCounter = downloadCounter + 1
			except:
				print("Fille Download Error :(")
				filename = filename + " - Download Error"
		else:
			filename = "No File Posted"
		
		with open(path + "/thread.csv", 'wb') as thread_file:
			thread_writer = csv.writer(thread_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_ALL)
			thread_writer.writerow([timeStamp, posterID, name, postID, subjectText, commentText, filename])
	
		j = j + 1

	
	#Iterate the loop
	i = i + 1
print ("Front Page Scrape Completed - " + str(downloadCounter) + " files downloaded")

BEGINNING 4CHAN FRONT PAGE SCRAPE
Current board: pol
Now parsing front page threads...
STARTING NEW THREAD - #124205675
Attempting to create directory for thread pol/124205675 - welcome-to-pol-politically-incorrect
Successfully created the directory pol/124205675 - welcome-to-pol-politically-incorrect
Now processing post 124205675
Fille Download Error :(
STARTING NEW THREAD - #259848258
Attempting to create directory for thread pol/259848258 - check-the-catalog-before-posting-a-new-thread-

Successfully created the directory pol/259848258 - check-the-catalog-before-posting-a-new-thread-

Now processing post 259848258
Fille Download Error :(
STARTING NEW THREAD - #379568139
Attempting to create directory for thread pol/379568139 - fuentes-has-officially-been-exposed-as-a
Successfully created the directory pol/379568139 - fuentes-has-officially-been-exposed-as-a
Now processing post 379568139
Fille Download Error :(
Now processing post 379568241
Now processing post 379568260
Now processin

In [9]:
!zip -r /content/file.zip /content/pol

  adding: content/pol/ (stored 0%)
  adding: content/pol/frontPage-2022-05-26 15:51:43.094885.csv (deflated 42%)
  adding: content/pol/379588465 - shes-right-you-know/ (stored 0%)
  adding: content/pol/379588465 - shes-right-you-know/thread.csv (deflated 35%)
  adding: content/pol/379587939 - chug-comfy-happening-in-ukraine-general-3330/ (stored 0%)
  adding: content/pol/379587939 - chug-comfy-happening-in-ukraine-general-3330/thread.csv (deflated 29%)
  adding: content/pol/379589092 - seemingly-executed-ukrainian-soldiers-supposedly/ (stored 0%)
  adding: content/pol/379589092 - seemingly-executed-ukrainian-soldiers-supposedly/thread.csv (deflated 8%)
  adding: content/pol/379587356 - parents-forced-to-wait-while-their-kids-died/ (stored 0%)
  adding: content/pol/379587356 - parents-forced-to-wait-while-their-kids-died/thread.csv (deflated 16%)
  adding: content/pol/379595386 - shock-russian-atrocities-in-bela-tserkva/ (stored 0%)
  adding: content/pol/379595386 - shock-russian-atroci

In [10]:
from google.colab import files
files.download("/content/file.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>